<a href="https://colab.research.google.com/github/nilsroemertum/synlab-slim-viewer/blob/master/notebooks/viewers_deployment/slim_Firebase_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Slim Deployment with Firebase and Google Cloud

This notebook complements the tutorial available in this document:
[Google Doc: Slim Deployment with Firebase and Google Cloud](https://tinyurl.com/idc-slim-gcp).
You need to complete the steps discussed in that document before proceeding with this notebook! Please use the document above for providing feedback or asking questions.



# Prerequisites

> **User interaction required**: In the following cell, set the variable gcpProjectID to your **Project ID**, gcpClientID to your **Client ID** and FirebaseProjectName to the **name of your Firebase project**.

In [1]:
gcpProjectID = 'synlab-pacs-workflow'
gcpClientID = '"62039621046-rt1fu02rnnfrlf2icjh2q1cmervka8au.apps.googleusercontent.com"'
FirebaseProjectName = 'synlab-pacs-slim-viewer'

## Install yarn

In [2]:
!npm install -g yarn

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 1 package in 2s
⠋

## Install Firebase command line tools

In [3]:
!curl -sL https://firebase.tools | bash

-- Checking for existing firebase-tools...
-- Checking your machine type...
-- Downloading binary from https://firebase.tools/bin/linux/latest
######################################################################## 100.0%
-- Setting permissions on binary... /usr/local/bin/firebase
-- Checking your PATH variable...
-- firebase-tools@14.16.0 is now installed
-- All Done!


## Clone Slim source code

In [39]:
!git clone https://github.com/nilsroemertum/synlab-slim-viewer.git

Cloning into 'synlab-slim-viewer'...
remote: Enumerating objects: 1886, done.
remote: Counting objects: 100% (1070/1070), done.
remote: Compressing objects: 100% (353/353), done.
remote: Total 1886 (delta 911), reused 717 (delta 717), pack-reused 816 (from 2)
Receiving objects: 100% (1886/1886), 33.82 MiB | 39.67 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.


## Install the node version management library (n) to update the node version to match Slim's

In [5]:
!npm install -g n

⠙⠹⠸⠼
added 1 package in 824ms
⠼

## Install and use node 14 using n



In [41]:
!n install 24 && n 24

     copying : node/24.8.0
   installed : v24.8.0 (with npm 11.6.0)
     copying : node/24.8.0
   installed : v24.8.0 (with npm 11.6.0)


# Configure and deploy Slim using Firebase


## Install Slim's dependencies using yarn



In [42]:
!cd synlab-slim-viewer && yarn

yarn install v1.22.22
warning ../package.json: No license field
[1/4] Resolving packages...
warning Resolution field "semver@7.5.2" is incompatible with requested version "semver@^7.6.0"
warning Resolution field "semver@7.5.2" is incompatible with requested version "semver@^7.5.3"
warning Resolution field "wrap-ansi@7.0.0" is incompatible with requested version "wrap-ansi@^8.1.0"
warning Resolution field "make-dir@3.1.0" is incompatible with requested version "make-dir@^2.1.0"
warning Resolution field "rollup@3.29.5" is incompatible with requested version "rollup@^2.43.1"
warning Resolution field "nth-check@2.0.1" is incompatible with requested version "nth-check@^1.0.2"
[2/4] Fetching packages...
[] 0/1728[] 95/1728[] 206/1728[] 304/1728[] 392/1728[] 480/1728[] 580/1728[] 672/1728[] 744/1728[] 840/1728[] 950/1728[] 1024/1728[] 1120/1728[] 1208/1728[] 1316/1728[] 1432/1728[] 1464/1728[] 1541/1728[] 1640/1728[] 1728/1728[3/4] Linking dependencies...
warning " > @babel/preset-env@7.20.2"

## Update Slim's configuration file
> The following code snipped configures things like OpenID Connect (OIDC) and server settings.



In [43]:
settings = """
window.config = {
  // This must match the location configured for web server
  path: '/',
  servers: [
    {
      id: 'example',
      // This must match the proxy location configured for the DICOMweb server
      url:"https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb",
      write: false
    }
  ],
 oidc: {
    authority: "https://accounts.google.com",
    clientId: %s,
    scope: "email profile openid https://www.googleapis.com/auth/cloud-healthcare",
    grantType: "implicit"
  },
  disableWorklist: false,
  disableAnnotationTools: false,
  enableServerSelection: true,
  mode: 'light',
  preload: true,
  annotations: [
    {
      finding: {
        value: '85756007',
        schemeDesignator: 'SCT',
        meaning: 'Tissue'
      },
      findingCategory: {
        value: '91723000',
        schemeDesignator: 'SCT',
        meaning: 'Anatomical structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 255, 0, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      },
    },
    {
      finding: {
        value: '108369006',
        schemeDesignator: 'SCT',
        meaning: 'Tumor'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 0, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '34823008',
        schemeDesignator: 'SCT',
        meaning: 'Tumor necrosis'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 51, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '369705002',
        schemeDesignator: 'SCT',
        meaning: 'Invasive tumor border'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['line', 'freehandline'],
      style: {
        stroke: {
          color: [51, 102, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '399721002',
        schemeDesignator: 'SCT',
        meaning: 'Tumor infiltration by lymphocytes present'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 204, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '47973001',
        schemeDesignator: 'SCT',
        meaning: 'Artifact'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 80, 80, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    }
  ]
}
"""%(gcpClientID)

with open("slim/public/config/example.js", "w") as f:
    f.write(settings)

## Update SLIM's Firebase config to deploy configuration



In [44]:
settings_firebase = """
{
  "hosting": {
    "public": "build",
    "ignore": [
      "firebase.json",
      "**/.*",
      "**/node_modules/**"
    ],
    "rewrites": [
      {
        "source": "**",
        "destination": "/index.html"
      }
    ]
  }
}
"""

with open("slim/firebase.json", "w") as f:
  f.write(settings_firebase)

## Build SLIM



In [45]:
!cd synlab-slim-viewer && yarn build

yarn run v1.22.22
warning ../package.json: No license field
$ craco build
(node:11604) [DEP0176] DeprecationWarning: fs.F_OK is deprecated, use fs.constants.F_OK instead
(Use `node --trace-deprecation ...` to show where the warning was created)
Creating an optimized production build...
Browserslist: caniuse-lite is outdated. Please run:
  npx update-browserslist-db@latest
  Why you should do it regularly: https://github.com/browserslist/update-db#readme
Browserslist: caniuse-lite is outdated. Please run:
  npx update-browserslist-db@latest
  Why you should do it regularly: https://github.com/browserslist/update-db#readme
Compiled successfully.

File sizes after gzip:

  1.15 MB    build/static/js/748.b2400262.chunk.js
  700.84 kB  build/static/js/dicomMicroscopyViewer.min.js
  102.57 kB  build/static/js/main.93037a91.js
  73.12 kB   build/static/css/808.25d876da.chunk.css
  71.59 kB   build/static/css/912.c78c3af2.chunk.css
  71.51 kB   build/static/js/dataLoader.worker.min.js
  45.33 

## Login and initialize Firebase configuration

  

> **User interaction required**: In the next step we login to Firebase and configure the setup of Firebase on the VM. When prompted regarding error reporting information, double click next to the prompt, and type your selection. Next, follow the authorization URL to grant the necessary permissions to Firebase, then double-click next to the "Paste authorization code", paste the code and hit "Enter" to complete the process.

In [46]:
!firebase login --no-localhost

i  The Firebase CLI’s MCP server feature can optionally make use of Gemini in Firebase. Learn more about Gemini in Firebase and how it uses your data: https://firebase.google.com/docs/gemini-in-firebase#how-gemini-in-firebase-uses-your-data
? Enable Gemini in Firebase features? (Y/n)? Enable Gemini in Firebase features? (Y/n) Y✔ Enable Gemini in Firebase features? Yes

i  Firebase optionally collects CLI and Emulator Suite usage and error reporting information to help improve our products. Data is collected in accordance with Google's privacy policy (https://policies.google.com/privacy) and is not used to identify you.
? Allow Firebase to collect CLI and Emulator Suite usage and error reporting 
information? (Y/n)? Allow Firebase to collect CLI and Emulator Suite usage and error reporting 
information? (Y/n) >✔ Allow Firebase to collect CLI and Emulator Suite usage and error reporting 
information? Yes

i  To change your preferences at any time, run `firebase logout` and `firebase logi

> **User interaction required**: Here you need to login to your Google account where you will be prompted to follow an authorization URL to grant the necessary permissions. Afterwards, we set the default GCP project ID to the one you chose at the beginning of this notebook.   

In [47]:
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Hp0MYki05T8AMymH4j8gGwF1J9FzAq&prompt=consent&token_usage=remote&access_type=offline&code_challenge=4oA8IkjTKwCaVgxwVvjTlSLKXGNTHtk3irpzpUv8KOo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVGzR1AJv2OKkSUc3qIe4QN6vK9CdppNNI6suldwJCHGd2cicYlazH43A5NSKx98rwSa_A

You are now logged in as [nilsroemer@yahoo.com].
Your current project i

In [48]:
!gcloud config set project $gcpProjectID
!gcloud config get project

Updated property [core/project].
synlab-pacs-workflow


> **User interaction required**: You should have created a Firebase project as a prerequisite step. Select a project or modify SLIM's .firebaserc file which sets the default firebase project.

In [49]:
!firebase projects:list

⠋ Preparing the list of your Firebase projects⠙ Preparing the list of your Firebase projects⠹ Preparing the list of your Firebase projects⠸ Preparing the list of your Firebase projects⠼ Preparing the list of your Firebase projects⠴ Preparing the list of your Firebase projects✔ Preparing the list of your Firebase projects
┌─────────────────────────┬─────────────────────────┬────────────────┬──────────────────────┐
│ Project Display Name    │ Project ID              │ Project Number │ Resource Location ID │
├─────────────────────────┼─────────────────────────┼────────────────┼──────────────────────┤
│ synlab-pacs-slim-viewer │ synlab-pacs-slim-viewer │ 584538855378   │ [Not specified]      │
└─────────────────────────┴─────────────────────────┴────────────────┴──────────────────────┘

1 project(s) total.


In [51]:
!cd synlab-slim-viewer && firebase use $FirebaseProjectName

Now using project synlab-pacs-slim-viewer


## Deploy
> Finally, we deploy the app. Do not close the notebook, keep the tab open and return to the Google documentation.

In [52]:
!cd synlab-slim-viewer && firebase deploy


=== Deploying to 'synlab-pacs-slim-viewer'...

i  deploying hosting
i  hosting[synlab-pacs-slim-viewer]: beginning deploy...
i  hosting[synlab-pacs-slim-viewer]: found 40 files in build
⠋ ⠙ ⠹ ⠸  hosting: hashing files [20/40] (50%)⠼  hosting: hashing files [20/40] (50%)⠴  hosting: hashing files [29/40] (72%)⠦  hosting: hashing files [29/40] (72%)⠧  hosting: hashing files [29/40] (72%)⠇  hosting: hashing files [35/40] (87%)⠏  hosting: hashing files [35/40] (87%)⠋  hosting: hashing files [36/40] (90%)⠙  hosting: hashing files [36/40] (90%)⠹  hosting: hashing files [36/40] (90%)⠸  hosting: hashing files [37/40] (92%)⠼  hosting: hashing files [37/40] (92%)⠴  hosting: hashing files [37/40] (92%)⠦  hosting: hashing files [37/40] (92%)⠧  hosting: hashing files [37/40] (92%)⠇  hosting: hashing files [38/40] (95%)⠏  hosting: hashing files [38/40] (95%)⠋  hosting: hashing files [38/40] (95%)⠙  hosting: hashing files [38/40] (95%)⠹  hosting: hashing files [38/40] (95%)⠸  hosting: hashing files [